In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient
from copy import deepcopy
import plotly.express as px

In [ ]:
# Connect to MongoDB
client = MongoClient('127.0.0.1', 27017)
db = client.frtp
collection = db.documents

In [ ]:
# Extract all the data available
result = collection.find()
df = pd.DataFrame(list(result))
df['year'] = pd.to_datetime(df['year'], format='%y')

In [ ]:
df.head(5)

In [ ]:
timestamps = df.year.to_list()

# Use unclean version of text
text = df.text.to_list()

In [ ]:
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(text, show_progress_bar=True)
topic_model = BERTopic(verbose=True,calculate_probabilities=True)
topics, probs = topic_model.fit_transform(text,embeddings=embeddings)
# topics, probs = topic_model.reduce_topics(text, topics, probabilities=probs, nr_topics=20)

In [ ]:
# Explore a topic
topic_model.get_topic(18)

In [ ]:
# Overview of the topics
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_topics()

# DTM

In [ ]:
# Running DTM on the entire dataset
topics_over_time = topic_model.topics_over_time(text, topics, timestamps)

In [ ]:
topics_over_time

In [ ]:
# First 10 topics bases on frequency
topics_over_time.head(10)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time,top_n_topics=10)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topic_evolution = pd.DataFrame()
timestamps_set = sorted(set(timestamps))
for timestamp in timestamps_set:
    temp_df = topics_over_time[topics_over_time['Timestamp'] == timestamp]
    if topic_evolution.shape[0] == 0:
        temp_df = temp_df[['Topic','Name','Words','Frequency']]
        temp_df = temp_df.rename(columns={'Frequency':str(timestamp)})
        topic_evolution = deepcopy(temp_df)
    else:
        temp_df = temp_df[['Topic','Frequency']]
        temp_df = temp_df.rename(columns={'Frequency':str(timestamp)})
        topic_evolution = topic_evolution.merge(temp_df,on='Topic',how='outer')

In [ ]:
topic_evolution = topic_evolution.fillna(0)

In [ ]:
topic_evolution.head(10)

In [ ]:
columns_to_process = topic_evolution.columns.to_list()
columns_to_process.remove('Topic')
columns_to_process.remove('Name')
columns_to_process.remove('Words')
columns_to_process

In [ ]:
for index in range(0,len(columns_to_process)-1):
    new_column = columns_to_process[index+1].split('-')[0] + '-' + columns_to_process[index].split('-')[0]
    topic_evolution[new_column] = topic_evolution[columns_to_process[index+1]] - topic_evolution[columns_to_process[index]]
topic_evolution.head(10)

In [ ]:
topics = topics_over_time.Topic.unique()
change_in_topics = pd.DataFrame()
for topic in topics:
    topic_df = topics_over_time[topics_over_time['Topic'] == topic]
    topic_df = topic_df.sort_values('Timestamp')
    topic_df['Previous_Frequency'] = topic_df.Frequency.shift(1)
    topic_df['Change'] = topic_df['Frequency'] - topic_df['Previous_Frequency']
    max_freq = max(topic_df['Frequency'])
    topic_df['%_Change'] = topic_df['Change']*100/max_freq
    change_in_topics = change_in_topics.append(topic_df,ignore_index=True)
change_in_topics = change_in_topics[change_in_topics['Topic'] != -1]
change_in_topics = change_in_topics.dropna()
change_in_topics.head(10)

In [ ]:
fig = px.line(change_in_topics, x="Timestamp", y="%_Change",color='Words', title='YOY change in topic frequency')
fig.show()